
# 分類における一般化

:label: `chap_classification_generalization`

これまで、複数の出力とソフトマックス関数を使用して (線形) ニューラル ネットワークをトレーニングすることによって、マルチクラス分類の問題に取り組む方法に焦点を当ててきました。モデルの出力を確率的予測として解釈し、クロスエントロピー損失関数を動機付けて導き出しました。この関数は、モデル (パラメーターの固定セットについて) が実際のラベルに割り当てる負の対数尤度を計算します。そして最後に、モデルをトレーニング セットに適合させることで、これらのツールを実践します。ただし、いつものように、私たちの目標は、これまでに見たことのないデータ (テスト セット) に基づいて経験的に評価された、*一般的なパターン*を学習することです。トレーニングセットでの精度が高いことには何の意味もありません。各入力が一意である場合は常に (実際、これはほとんどの高次元データセットに当てはまります)、最初のトレーニング エポックでデータセットを記憶し、その後はラベルを参照するたびにラベルを検索するだけで、トレーニング セットで完全な精度を達成できます。新しいイメージ。それでも、正確なトレーニング サンプルに関連付けられた正確なラベルを記憶しても、新しいサンプルを分類する方法はわかりません。さらなるガイダンスがなければ、新しい例に遭遇するたびにランダムな推測に頼らざるを得なくなるかもしれません。

多くの深刻な質問には、すぐに対応する必要があります。
1. 基礎となる母集団に対する分類器の精度を正確に推定するには、いくつのテスト例が必要ですか?
1. 同じテストでモデルを繰り返し評価し続けるとどうなるでしょうか?
1. なぜ、線形モデルをトレーニングセットに当てはめることが、単純な暗記スキームよりもうまくいくと期待する必要があるのでしょうか?

 :numref: `sec_generalization_basics`では、線形回帰のコンテキストにおける過学習と一般化の基本を紹介しましたが、この章ではもう少し深く掘り下げて、統計学習理論の基本的な考え方をいくつか紹介します。多くの場合、*先験的に*一般化を保証できることがわかります。多くのモデルについて、一般化ギャップ $\epsilon$ の任意の上限について、必要なサンプル数 $n$ を決定できることがよくあります。少なくとも $n$ サンプルの場合、経験的誤差は、*分布を生成するデータについて*、真の誤差の $\epsilon$ 以内に収まります。残念なことに、この種の保証は一連の深い知的構成要素を提供するものの、深層学習の実践者にとって実用性が限られていることが判明しました。要するに、これらの保証は、ディープ ニューラル ネットワークが*一般に、*私たちが関心のあるタスクに関して、通常、はるかに優れた一般化を行うことがわかったとしても、ディープ ニューラル ネットワークのアプリオリな一般化を保証するには、途方もない数の例 (おそらく数兆以上) が必要であることを示唆しています。例が少なくなります (数千)。したがって、深層学習の実践者は、アプリオリな保証を完全に放棄することが多く、代わりに、過去に同様の問題についてうまく一般化したことに基づいて手法を採用し、経験的評価を通じて一般化を*事後的に*証明します。 :numref: `chap_perceptrons`に到達したら、一般化を再検討し、ディープ ニューラル ネットワークが実際に一般化する理由を説明するために出てきた膨大な科学文献を簡単に紹介します。

## テストセット

私たちはすでに汎化誤差を評価するためのゴールドスタンダード方法としてテストセットに依存し始めているので、そのような誤差推定の特性について議論することから始めましょう。どのように取得されたかは気にせずに、固定分類子 $f$ に注目してみましょう。さらに、例 $\mathcal{D} = {(\mathbf{x}^{(i)},y^{(i)})}_{i=1}^n$ の*新しい*データセットを所有しているとします。分類子 $f$ のトレーニングには使用されませんでした。 $\mathcal{D}$ に対する分類子 $f$ の*経験的誤差は*、単に予測 $f(\mathbf{x}^{(i)})$ が真のラベル $y と一致しないインスタンスの割合です。 ^{(i)}$ であり、次の式で与えられます。

 $$\epsilon_\mathcal{D}(f) = \frac{1}{n}\sum_{i=1}^n \mathbf{1}(f(\mathbf{x}^{(i)}) \neq y^{(i)}).$$

対照的に、*母集団誤差は*、分類器が一致しない、基礎となる母集団 (確率密度関数 $p(\mathbf{x},y)$ によって特徴付けられるある分布 $P(X,Y)$) 内の例の*期待される*割合です。真のラベルの場合:

 $$\epsilon(f) = E_{(\mathbf{x}, y) \sim P} \mathbf{1}(f(\mathbf{x}) \neq y) = \int\int \mathbf{1 }(f(\mathbf{x}) \neq y) p(\mathbf{x}, y) ;d\mathbf{x} dy.$$

 $\epsilon(f)$ は私たちが実際に関心のある量ですが、すべての人を測定せずに大集団の平均身長を直接観察できないのと同様に、それを直接観察することはできません。この数量はサンプルに基づいてのみ推定できます。検定セット $\mathcal{D}$ は基礎となる母集団を統計的に表すため、$\epsilon_\mathcal{D}(f)$ を母集団誤差 $\epsilon(f)$ の統計的推定量として見ることができます。さらに、関心量 $\epsilon(f)$ は (確率変数 $\mathbf{1}(f(X) \neq Y)$ の) 期待値であり、対応する推定量 $\epsilon_\mathcal{D }(f)$ は標本平均であり、母集団誤差の推定は、 :numref: `sec_prob`から思い出される平均推定の古典的な問題にすぎません。

*中心極限定理*と呼ばれる確率論からの重要な古典的な結果は、次のように、平均 $\mu$ と標準偏差 $\sigma$ を持つ任意の分布から抽出された $n$ 個のランダム サンプル $a_1, ..., a_n$ を所有することを保証します。サンプル数 $n$ が無限大に近づくと、サンプル平均 $\hat{\mu}$ は、真の平均を中心とし、標準偏差 $\sigma/\sqrt{n}$ を持つ正規分布に近似する傾向があります。これですでに重要なことがわかります。例の数が増えるにつれて、テスト誤差 $\epsilon_\mathcal{D}(f)$ は $\mathcal の割合で真の誤差 $\epsilon(f)$ に近づくはずです。 {O}(1/\sqrt{n})$。したがって、テスト誤差を 2 倍の精度で推定するには、4 倍の大きさのテスト セットを収集する必要があります。テスト誤差を 100 分の 1 に減らすには、1 万倍の大きさのテスト セットを収集する必要があります。一般に、このような $\mathcal{O}(1/\sqrt{n})$ の割合は、統計的に期待できる最高の値であることがよくあります。

テスト誤差 $\epsilon_\mathcal{D}(f)$ が真の誤差 $\epsilon(f)$ に収束する漸近率について何かがわかったので、いくつかの重要な詳細に焦点を当てることができます。対象となる確率変数 $\mathbf{1}(f(X) \neq Y)$ は値 $0$ と $1$ のみを取ることができるため、ベルヌーイ確率変数であり、それが取る確率を示すパラメーターによって特徴付けられることを思い出してください。値 $1$。ここで、$1$ は分類器がエラーを起こしたことを意味するため、確率変数のパラメーターは実際には真のエラー率 $\epsilon(f)$ になります。ベルヌーイの分散 $\sigma^2$ は、式 $\epsilon(f)(1-\epsilon(f))$ に従って、そのパラメータ (ここでは $\epsilon(f)$) に依存します。 $\epsilon(f)$ は最初は不明ですが、$1$ を超えることはできないことがわかっています。この関数を少し調査すると、真の誤り率が $0.5$ に近い場合に分散が最も高く、$0$ または $1$ に近い場合にははるかに低くなる可能性があることがわかります。これは、誤差 $\epsilon(f)$ の推定 $\epsilon_\mathcal{D}(f)$ の漸近標準偏差 ($n$ のテスト サンプルの選択による) が以下を超えることはできないことを示しています。 $\sqrt{0.25/n}$。

この比率が有限のサンプルを所有する場合ではなく、テスト セットのサイズが無限に近づくときの動作を特徴付けるという事実を無視すると、テスト誤差 $\epsilon_\mathcal{D}(f)$ を近似させたい場合に次のことがわかります。母集団誤差 $\epsilon(f)$ が 1 つの標準偏差が $\pm 0.01$ の間隔に相当する場合、およそ 2500 個のサンプルを収集する必要があります。 2 つの標準偏差をその範囲に当てはめて $\epsilon_\mathcal{D}(f) \in \epsilon(f) \pm 0.01$ の 95% にしたい場合は、10000 個のサンプルが必要になります。

これは、機械学習の多くの一般的なベンチマークのテスト セットのサイズであることがわかります。毎年何千もの深層学習の応用論文が出版され、エラー率が 0.01 ドル以下で大幅に改善されていると知ったら驚かれるかもしれません。もちろん、エラー率が $0$ にはるかに近い場合、$0.01$ の改善は実際に大きな問題になる可能性があります。

これまでの分析の厄介な特徴の 1 つは、実際には漸近性、つまりサンプル サイズが無限大になるにつれて $\epsilon_\mathcal{D}$ と $\epsilon$ の関係がどのように変化するかについてしか教えていないことです。幸いなことに、確率変数には制限があるため、Hoeffding (1963) による不等式を適用することで有効な有限サンプル範囲を取得できます。

 $$P(\epsilon_\mathcal{D}(f) - \epsilon(f) \geq t) &lt; \exp\left( - 2n t^2 \right).$$

推定値 $\epsilon_\mathcal{D}(f)$ と真の誤り率 $\epsilon(f)$ の間の距離 $t$ が 95% の信頼度で結論付けられる最小のデータセット サイズを求める解です。 $0.01$ を超えない場合、上記の漸近分析で提案された $10000$ のサンプルと比較して、約 $15000$ のサンプルが必要であることがわかります。統計を詳しく調べてみると、この傾向が一般的に当てはまることがわかります。有限サンプルでも成立する保証は、通常、もう少し保守的です。物事の仕組みでは、これらの数字はそれほど離れていないことに注意してください。これは、法廷に持ち込める保証はなくても、おおよその数字を得る漸近分析の一般的な有用性を反映しています。

## テストセットの再利用

ある意味、これで機械学習の実証研究を成功させる準備が整いました。ほぼすべての実用的なモデルは、テスト セットのパフォーマンスに基づいて開発および検証されており、これであなたはテスト セットのマスターになります。固定分類子 $f$ については、そのテスト誤差 $\epsilon_\mathcal{D}(f)$ を評価することがわかっており、その母集団誤差 $\epsilon(f)$ について何が言えるか (何が言えないか) を正確に知っています。 。

したがって、この知識を活用して、最初のモデル $f_1$ をトレーニングする準備をしているとします。分類器の誤り率のパフォーマンスにどの程度自信を持たせる必要があるかがわかったら、上記の分析を適用して、テスト セットに確保する適切な例の数を決定します。さらに、:numref: `sec_generalization_basics`からの教訓を心に留め、事前分析、ハイパーパラメータ調整、さらには検証セット上の複数の競合するモデル アーキテクチャからの選択をすべて実行することで、テスト セットの神聖性を確実に保持したと仮定します。 。最後に、テスト セットでモデル $f_1$ を評価し、関連する信頼区間を使用して母集団誤差の不偏推定値をレポートします。

これまでのところ、すべてが順調に進んでいるように見えます。しかし、その夜、あなたは午前 3 時に目覚め、新しいモデリング アプローチの素晴らしいアイデアを思いつきました。翌日、新しいモデルをコード化し、検証セットでそのハイパーパラメータを調整すると、新しいモデル $f_2$ が動作するようになるだけでなく、エラー率が $f_1$ よりも大幅に低くなることがわかります。しかし、最終評価の準備をしていると、発見のスリルは突然薄れてしまいます。お試しセットがありません!

元のテスト セット $\mathcal{D}$ はまだサーバー上にありますが、ここで 2 つの手強い問題に直面しています。まず、テスト セットを収集したときに、単一の分類子 $f$ を評価しているという仮定の下で、必要な精度レベルを決定しました。ただし、同じテスト セットで複数の分類子 $f_1、...、f_k$ を評価することに取り組む場合は、誤検出の問題を考慮する必要があります。以前は、単一の分類子 $f$ に対して $\epsilon_\mathcal{D}(f) \in \epsilon(f) \pm 0.01$ であると 95% 確信していたため、誤解を招く結果が生じる確率はほんのわずかでした。 5%。 $k$ 分類子が混在しているため、その中にテスト セットのパフォーマンスが誤解を招くものがないことを保証するのは困難です。 20 個の分類子が検討されているため、そのうちの少なくとも 1 つが誤解を招くスコアを受け取った可能性を排除する力はまったくないかもしれません。この問題は複数の仮説検定に関連しており、統計に関する膨大な文献があるにもかかわらず、依然として科学研究を悩ませている永続的な問題です。

それで心配するのに十分でない場合は、その後の評価で得られる結果に不信感を抱く特別な理由があります。テスト セットのパフォーマンスの分析は、テスト セットとの接触がない状態で分類器が選択されたという仮定に基づいていたため、テスト セットは基礎となる母集団からランダムに抽出されたものとみなすことができることを思い出してください。ここでは、複数の関数をテストしているだけでなく、$f_1$ のテスト セットのパフォーマンスを観察した後、後続の関数 $f_2$ が選択されています。テスト セットからの情報がモデラーに漏洩すると、それは最も厳密な意味で真のテスト セットに戻ることはできません。この問題は*適応過学習*と呼ばれ、学習理論家や統計学者にとって非常に興味深いトピックとして最近浮上しています:cite: `dwork2015preserving` 。幸いなことに、ホールドアウト セットからすべての情報が漏洩する可能性はあり、理論上の最悪のシナリオは暗いですが、これらの分析は保守的すぎる可能性があります。実際には、実際のテスト セットを作成し、それらを参照する頻度をできるだけ低くし、信頼区間を報告するときに複数の仮説検定を考慮し、賭け金が高くデータセット サイズが小さい場合には、より積極的に警戒を強化するように注意してください。一連のベンチマーク チャレンジを実行する場合、多くの場合、ラウンドごとに古いテスト セットを検証セットに降格できるように、いくつかのテスト セットを維持することをお勧めします。

## 統計的学習理論

一見すると、*私たちが実際に持っているのはテスト セットだけです*が、この事実は奇妙に満足できないように思えます。まず、私たちが*真のテスト セットを*所有していることはめったにありません。私たちがデータセットを作成しているのでなければ、おそらく他の誰かが私たちの表向きの「テスト セット」に対して独自の分類器をすでに評価しているでしょう。そして、最初に失敗したときでさえ、私たちはすぐにイライラして、自分の数字を信頼できないという苦痛を感じずに、その後のモデリングの試みを評価できればいいのにと思います。さらに、真のテストセットであっても、分類器が実際に母集団に一般化したかどうかを*事後的に*知ることができるだけであり、それが一般化するはずであると*先験的*に期待する理由があるかどうかを知ることはできません。

これらの不安を念頭に置くことで、*統計学習理論*の魅力を十分に理解できるようになったかもしれません。統計学習理論は機械学習の数学的サブ分野であり、その実践者は、経験的データに基づいて訓練されたモデルがなぜ一般化できるのか、いつ一般化できるのかを説明する基本原理を解明することを目指しています。目に見えないデータ。数十年にわたる統計学習研究者の主な目的の 1 つは、モデル クラスのプロパティとデータセット内のサンプル数を関連付けて、一般化ギャップを制限することでした。

学習理論家は、学習セット $f_\mathcal{S}$ の学習済み分類子 $\epsilon_\mathcal{S}(f_\mathcal{S})$ の*経験的誤差*$\epsilon_\mathcal{S}(f_\mathcal{S})$ 間の差を制限することを目指しています。 \mathcal{S}$、および基礎となる母集団に対する同じ分類器の真の誤差 $\epsilon(f_\mathcal{S})$。これは、先ほど取り上げた評価の問題に似ているように見えるかもしれませんが、大きな違いがあります。以前は、分類子 $f$ は修正されており、評価目的でのみデータセットが必要でした。そして実際、固定分類子は一般化します。(これまでに見たことのない) データセットでの誤差は、母集団誤差の不偏推定値です。しかし、分類器が同じデータセットでトレーニングおよび評価される場合、何が言えるでしょうか?トレーニング誤差がテスト誤差に近いと確信できるでしょうか?

学習した分類子 $f_\mathcal{S}$ は、事前に指定された関数セット $\mathcal{F}$ の中から選択する必要があるとします。テスト セットについての説明を思い出してください。単一の分類子の誤差を推定するのは簡単ですが、分類子のコレクションを検討し始めると、物事は複雑になります。たとえ 1 つの (固定された) 分類器の経験的誤差が高い確率で真の誤差に近いとしても、分類器のコレクションを考慮すると、セット内の*1 つの分類器だけが*ひどく誤った推定を受ける可能性について心配する必要があります。エラー。心配なのは、コレクション内の 1 つの分類器だけが誤解を招くほど低い誤差を受け取った場合、それを選択してしまい、母集団誤差を大幅に過小評価してしまう可能性があることです。さらに、線形モデルの場合でも、そのパラメーターは継続的に値が計算されるため、通常は無限のクラスの関数 ($|\mathcal{F}| = \infty$) の中から選択することになります。

この問題に対する野心的な解決策の 1 つは、一様収束を証明するための分析ツールを開発することです。つまり、クラス $f\in\mathcal{F}$ 内のすべての分類器の経験的誤差率が高い確率で*同時に*真の誤差に収束するということです。レート。言い換えれば、我々は、少なくとも $1-\delta$ (いくつかの小さな $\delta$ の場合) の確率で (すべての分類器の中で) 分類器の誤り率 $\epsilon(f)$ が存在しないと言える理論的原理を求めています。クラス $\mathcal{F}$) は、少量 $\alpha$ 以上誤って推定されます。明らかに、すべてのモデル クラス $\mathcal{F}$ に対してそのようなステートメントを作成することはできません。常に経験的誤差 $0$ を達成するが、基礎となる母集団に対するランダムな推測を上回るパフォーマンスを発揮しない記憶機械のクラスを思い出してください。

ある意味、暗記者のクラスは柔軟性が高すぎます。このような均一な収束結果が成立する可能性はありません。一方、固定分類子は役に立ちません。固定分類子は完全に一般化しますが、トレーニング データにもテスト データにも適合しません。したがって、学習の中心的な問題は、歴史的に、トレーニング データによりよく適合するが過適合のリスクがある、より柔軟な (分散が高い) モデル クラスと、一般化は良好だが過小適合のリスクがあるより厳格な (バイアスが高い) モデル クラスの間のトレードオフとして組み立てられてきました。学習理論における中心的な問題は、モデルがこのスペクトルのどこに位置するかを定量化し、関連する保証を提供するための適切な数学的分析を開発することでした。

一連の独創的な論文で、Vapnik と Chervonenkis は、相対周波数の収束に関する理論を関数のより一般的なクラスに拡張しました: 引用: `VapChe64,VapChe68,VapChe71,VapChe74b,VapChe81,VapChe91` 。この一連の作業の主な貢献の 1 つは、モデル クラスの複雑さ (柔軟性) (の概念の 1 つ) を測定する Vapnik-Chervonenkis (VC) 次元です。さらに、彼らの重要な結果の 1 つは、VC 次元とサンプル数の関数として、経験誤差と母集団誤差の差を制限します。

 $$P\left(R[p, f] - R_\mathrm{emp}[\mathbf{X}, \mathbf{Y}, f] &lt; \alpha\right) \geq 1-\delta \ \text{ }\ \alpha \geq c \sqrt{(\mathrm{VC} - \log \delta)/n}.$$ の場合

ここで、$\delta &gt; 0$ は境界に違反する確率、$\alpha$ は汎化ギャップの上限、$n$ はデータセットのサイズです。最後に、$c &gt; 0$ は、発生する可能性のある損失の規模にのみ依存する定数です。境界の使用法の 1 つは、$\delta$ と $\alpha$ の必要な値を代入して、収集するサンプルの数を決定することです。 VC 次元は、任意の (バイナリ) ラベルを割り当てることができるデータ ポイントの最大数を定量化し、それぞれに対してそのラベルと一致するクラス内のモデル $f$ を見つけます。たとえば、$d$ 次元の入力の線形モデルは VC 次元 $d+1$ を持ちます。線は 2 次元の 3 つの点に任意のラベルを割り当てることができますが、4 つの点には割り当てることができないことが簡単にわかります。残念ながら、この理論はより複雑なモデルに対して過度に悲観的になる傾向があり、この保証を得るには通常、望ましいエラー率を達成するために実際に必要なサンプルよりもはるかに多くのサンプルが必要になります。モデル クラスと $\delta$ を修正すると、エラー率が通常の $\mathcal{O}(1/\sqrt{n})$ 率で再び減衰することにも注意してください。 $n$ に関してこれ以上の成果を上げることはできそうにありません。ただし、モデル クラスを変更すると、VC 次元が一般化ギャップの悲観的な図を示す可能性があります。

## まとめ

モデルを評価する最も簡単な方法は、これまでに見たことのないデータで構成されるテスト セットを参照することです。テスト セットの評価は、真の誤差の不偏推定値を提供し、テスト セットが大きくなるにつれて望ましい $\mathcal{O}(1/\sqrt{n})$ レートに収束します。正確な漸近分布に基づいた近似信頼区間、または (より保守的な) 有限サンプル保証に基づいた有効な有限サンプル信頼区間を提供できます。実際、テスト セットの評価は、現代の機械学習研究の基盤です。ただし、テスト セットが真のテスト セットであることはほとんどありません (複数の研究者が繰り返し使用します)。同じテスト セットを使用して複数のモデルを評価すると、誤検出を制御することが困難になる場合があります。これは理論的には大きな問題を引き起こす可能性があります。実際には、問題の重要性は、問題のホールドアウト セットのサイズと、それらが単にハイパーパラメータの選択に使用されているのか、それともより直接的に情報が漏洩しているのかによって異なります。それにもかかわらず、実際のテスト セット (または複数) を厳選し、それらの使用頻度をできるだけ控えめにすることをお勧めします。

より満足のいく解決策を提供することを期待して、統計学習理論家は、モデル クラスにわたる一様な収束を保証する方法を開発しました。実際、すべてのモデルの経験的誤差が同時に真の誤差に収束する場合、ホールドアウト データに対しても同様に良好なパフォーマンスを示すことがわかっているため、トレーニング エラーを最小限に抑えて最高のパフォーマンスを発揮するモデルを自由に選択できます。重要なのは、そのような結果はいずれもモデル クラスの何らかのプロパティに依存する必要があることです。 Vladimir Vapnik と Alexey Chernovenkis は VC 次元を導入し、VC クラスのすべてのモデルに当てはまる均一な収束結果を示しました。クラス内のすべてのモデルの学習誤差は (同時に) 真の誤差に近づくことが保証され、$\mathcal{O}(1/\sqrt{n})$ のレートで近づくことが保証されます。 VC 次元の革命的な発見に続いて、多数の代替的な複雑さの尺度が提案されており、それぞれが同様の一般化の保証を容易にします。関数の複雑さを測定するいくつかの高度な方法の詳細については、:citet: `boucheron2005theory`を参照してください。残念ながら、これらの複雑さの尺度は統計理論において広く有用なツールとなっていますが、ディープ ニューラル ネットワークが一般化する理由を説明するには (そのまま適用すると) 無力であることが判明しました。ディープ ニューラル ネットワークには多くの場合、数百万 (またはそれ以上) のパラメーターがあり、ポイントの大規模なコレクションにランダムなラベルを簡単に割り当てることができます。それにもかかわらず、それらは実際の問題をよく一般化し、驚くべきことに、より大きな VC 次元を伴うにもかかわらず、より大きく深い場合には、よりよく一般化することがよくあります。次の章では、深層学習の文脈で一般化について再検討します。

## 演習
1. 固定モデル $f$ の誤差を $0.0001$ 以内で 99.9% 以上の確率で推定したい場合、サンプルはいくつ必要ですか?
1. 他の誰かがラベル付きテスト セット $\mathcal{D}$ を所有し、ラベルなしの入力 (特徴) のみを利用できるようにするとします。ここで、ラベルのない入力のそれぞれに対してモデル $f$ (モデル クラスに制限はありません) を実行し、対応するエラー $\epsilon_\mathcal{D}(f)$ を受け取ることによってのみテスト セットのラベルにアクセスできるとします。 。テスト セット全体をリークして、実際のエラーに関係なく、エラー $0$ があるように見える前に、いくつのモデルを評価する必要があるでしょうか?
1.  $5^\mathrm{th}$ 次多項式のクラスの VC 次元は何ですか?
1.  2 次元データ上の軸に揃えられた長方形の VC 次元はどれくらいですか?

[ディスカッション](https://discuss.d2l.ai/t/6829)
